In [27]:
from extractor import get_text
from scan import get_files
import pandas as pd
import os
import json
from prompt import *
from pathlib import Path
from tqdm.auto import tqdm
import requests
import re
from pathlib import Path

In [28]:
path_input = r"C:\Users\alika\OneDrive\Masaüstü\Profiler\FileProfiler\test\wiki_docs"

In [54]:
rows = []    
c = 0
for current_root, dirs, files in os.walk(path_input):
    for name in files:
        f = Path(os.path.join(current_root, name))
        rows.append({
        "Full Path": f,
        "File Name": f.stem,
        "Extension": f.suffix.lstrip('.')
        })
        c += 1

In [55]:
pd.DataFrame(rows)

,Full Path,File Name,Extension
0,C:\Users\alika\OneDrive\Masaüstü\Profiler\File...,List_of_Pakistani_films_of_2017,docx
1,C:\Users\alika\OneDrive\Masaüstü\Profiler\File...,Photosynthesis_system,docx
2,C:\Users\alika\OneDrive\Masaüstü\Profiler\File...,Por_Ese_Hombre,docx
3,C:\Users\alika\OneDrive\Masaüstü\Profiler\File...,Salix_atrocinerea,docx
4,C:\Users\alika\OneDrive\Masaüstü\Profiler\File...,Wee1-like_protein_kinase,docx


In [17]:
a = 'xxx.docx'

In [26]:
Path(a).suffix

'.docx'

In [24]:
a[3:8]

'.docx'

In [12]:
headers = {"Authorization": f"Bearer {os.getenv("OPENROUTER_KEY")}",
            "Content-Type": "application/json",}

data = {"model": "qwen/qwen3-8b",
        "messages": [{"role": "user", "content": sp},
                        {"role": "user", "content": up.format(text, question)}],}

r = requests.post(os.getenv("OPENROUTER_URL"),
                headers=headers, data=json.dumps(data))
if r.status_code==200:
        resp = json.loads(r.text.strip())["choices"][0]["message"]["content"]